In [22]:
import gc
import re
import json
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

In [52]:
IN_TRAIN = 'in/train-pruned.csv'
IN_TEST = 'in/test-pruned.csv'

OUT_TRAIN = 'in/train-wrangled.csv'
OUT_TEST = 'in/test-wrangled.csv'

In [3]:
train_df = pd.read_csv(IN_TRAIN, dtype={'fullVisitorId': 'str'}, low_memory=False)
train_df.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.deviceCategory,...,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue,totals.visits,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.isTrueDirect,trafficSource.medium,trafficSource.source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,desktop,...,1.0,1,1.0,1.0,NaN,1,NaN,NaN,organic,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,desktop,...,1.0,1,1.0,1.0,NaN,1,NaN,NaN,organic,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,desktop,...,1.0,1,1.0,1.0,NaN,1,NaN,NaN,organic,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,desktop,...,1.0,1,1.0,1.0,NaN,1,NaN,NaN,organic,google
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,mobile,...,1.0,1,NaN,1.0,NaN,1,NaN,True,organic,google


In [4]:
test_df = pd.read_csv(IN_TEST, dtype={'fullVisitorId': 'str'}, low_memory=False)
test_df.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.deviceCategory,...,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.visits,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.isTrueDirect,trafficSource.medium,trafficSource.source
0,Organic Search,20171016,6167871330617112363,6167871330617112363_1508151024,Not Socially Engaged,1508151024,2,1508151024,Chrome,desktop,...,Southeast Asia,NaN,4,NaN,4.0,1,NaN,True,organic,google
1,Organic Search,20171016,0643697640977915618,0643697640977915618_1508175522,Not Socially Engaged,1508175522,1,1508175522,Chrome,desktop,...,Southern Europe,NaN,5,1.0,5.0,1,NaN,NaN,organic,google
2,Organic Search,20171016,6059383810968229466,6059383810968229466_1508143220,Not Socially Engaged,1508143220,1,1508143220,Chrome,desktop,...,Western Europe,NaN,7,1.0,7.0,1,NaN,NaN,organic,google
3,Organic Search,20171016,2376720078563423631,2376720078563423631_1508193530,Not Socially Engaged,1508193530,1,1508193530,Safari,mobile,...,Northern America,NaN,8,1.0,4.0,1,NaN,NaN,organic,google
4,Organic Search,20171016,2314544520795440038,2314544520795440038_1508217442,Not Socially Engaged,1508217442,1,1508217442,Safari,desktop,...,Northern America,NaN,9,1.0,4.0,1,NaN,NaN,organic,google


In [5]:
LABEL = 'totals.transactionRevenue'

In [28]:
def true_false_cols(df, cols):
    for col in cols:
        df[col] = df[col].apply((lambda x: 1 if x == 'TRUE' else 0))
    return df

cols_to_true_false = ['device.isMobile', 'trafficSource.isTrueDirect',
                     'trafficSource.adwordsClickInfo.isVideoAd']

train_df_true_false = true_false_cols(train_df_onehot, cols_to_true_false)
train_df_true_false.head()

In [30]:
test_df_true_false = true_false_cols(test_df_onehot, cols_to_true_false)
test_df_true_false.head()

,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,device.browser,device.isMobile,geoNetwork.city,geoNetwork.country,...,device.operatingSystem Windows Phone,device.operatingSystem Xbox,device.operatingSystem iOS,geoNetwork.continent (not set),geoNetwork.continent Africa,geoNetwork.continent Americas,geoNetwork.continent Asia,geoNetwork.continent Europe,geoNetwork.continent Oceania,socialEngagementType Not Socially Engaged
0,20171016,6167871330617112363,6167871330617112363_1508151024,1508151024,2,1508151024,Chrome,0,(not set),Singapore,...,0,0,0,0,0,0,1,0,0,1
1,20171016,0643697640977915618,0643697640977915618_1508175522,1508175522,1,1508175522,Chrome,0,Zaragoza,Spain,...,0,0,0,0,0,0,0,1,0,1
2,20171016,6059383810968229466,6059383810968229466_1508143220,1508143220,1,1508143220,Chrome,0,not available in demo dataset,France,...,0,0,0,0,0,0,0,1,0,1
3,20171016,2376720078563423631,2376720078563423631_1508193530,1508193530,1,1508193530,Safari,0,Mountain View,United States,...,0,0,1,0,0,1,0,0,0,1
4,20171016,2314544520795440038,2314544520795440038_1508217442,1508217442,1,1508217442,Safari,0,San Jose,United States,...,0,0,0,0,0,1,0,0,0,1


In [25]:
def one_hot_encode_col(df, col):
    one_hot_encoded_cols = pd.get_dummies(df[col], prefix=col, prefix_sep=' ')
    df_without_original_col = df.drop(col, axis=1)
    return pd.concat([df_without_original_col, one_hot_encoded_cols], axis=1, sort=False)

def one_hot_encode_cols(df, cols):
    for col in cols:
        df = one_hot_encode_col(df, col)
    return df

cols_to_one_hot_encode = ['channelGrouping', 'device.deviceCategory', 'device.operatingSystem',
                         'geoNetwork.continent', 'socialEngagementType']

In [26]:
train_df_onehot = one_hot_encode_cols(train_df, cols_to_one_hot_encode)
train_df_onehot.head()

,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,device.browser,device.isMobile,geoNetwork.city,geoNetwork.country,...,device.operatingSystem Windows Phone,device.operatingSystem Xbox,device.operatingSystem iOS,geoNetwork.continent (not set),geoNetwork.continent Africa,geoNetwork.continent Americas,geoNetwork.continent Asia,geoNetwork.continent Europe,geoNetwork.continent Oceania,socialEngagementType Not Socially Engaged
0,20160902,1131660440785968503,1131660440785968503_1472830385,1472830385,1,1472830385,Chrome,False,Izmir,Turkey,...,0,0,0,0,0,0,1,0,0,1
1,20160902,377306020877927890,377306020877927890_1472880147,1472880147,1,1472880147,Firefox,False,not available in demo dataset,Australia,...,0,0,0,0,0,0,0,0,1,1
2,20160902,3895546263509774583,3895546263509774583_1472865386,1472865386,1,1472865386,Chrome,False,Madrid,Spain,...,0,0,0,0,0,0,0,1,0,1
3,20160902,4763447161404445595,4763447161404445595_1472881213,1472881213,1,1472881213,UC Browser,False,not available in demo dataset,Indonesia,...,0,0,0,0,0,0,1,0,0,1
4,20160902,27294437909732085,27294437909732085_1472822600,1472822600,2,1472822600,Chrome,True,not available in demo dataset,United Kingdom,...,0,0,0,0,0,0,0,1,0,1


In [27]:
test_df_onehot = one_hot_encode_cols(test_df, cols_to_one_hot_encode)
test_df_onehot.head()

,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,device.browser,device.isMobile,geoNetwork.city,geoNetwork.country,...,device.operatingSystem Windows Phone,device.operatingSystem Xbox,device.operatingSystem iOS,geoNetwork.continent (not set),geoNetwork.continent Africa,geoNetwork.continent Americas,geoNetwork.continent Asia,geoNetwork.continent Europe,geoNetwork.continent Oceania,socialEngagementType Not Socially Engaged
0,20171016,6167871330617112363,6167871330617112363_1508151024,1508151024,2,1508151024,Chrome,False,(not set),Singapore,...,0,0,0,0,0,0,1,0,0,1
1,20171016,0643697640977915618,0643697640977915618_1508175522,1508175522,1,1508175522,Chrome,False,Zaragoza,Spain,...,0,0,0,0,0,0,0,1,0,1
2,20171016,6059383810968229466,6059383810968229466_1508143220,1508143220,1,1508143220,Chrome,False,not available in demo dataset,France,...,0,0,0,0,0,0,0,1,0,1
3,20171016,2376720078563423631,2376720078563423631_1508193530,1508193530,1,1508193530,Safari,True,Mountain View,United States,...,0,0,1,0,0,1,0,0,0,1
4,20171016,2314544520795440038,2314544520795440038_1508217442,1508217442,1,1508217442,Safari,False,San Jose,United States,...,0,0,0,0,0,1,0,0,0,1


In [31]:
cols_to_minmax_scale = ['date', 'visitId', 'visitNumber',
                        'visitStartTime', 'totals.bounces',
                       'totals.hits', 'totals.newVisits',
                       'totals.pageviews', 'totals.visits']
scaler = MinMaxScaler()

train_df_true_false[cols_to_minmax_scale] = scaler.fit_transform(train_df_true_false[cols_to_minmax_scale])
train_df_true_false.head()

C:\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,device.browser,device.isMobile,geoNetwork.city,geoNetwork.country,...,device.operatingSystem Windows Phone,device.operatingSystem Xbox,device.operatingSystem iOS,geoNetwork.continent (not set),geoNetwork.continent Africa,geoNetwork.continent Americas,geoNetwork.continent Asia,geoNetwork.continent Europe,geoNetwork.continent Oceania,socialEngagementType Not Socially Engaged
0,0.0101,1131660440785968503,1131660440785968503_1472830385,0.088405,0.000000,0.088405,Chrome,0,Izmir,Turkey,...,0,0,0,0,0,0,1,0,0,1
1,0.0101,377306020877927890,377306020877927890_1472880147,0.089979,0.000000,0.089979,Firefox,0,not available in demo dataset,Australia,...,0,0,0,0,0,0,0,0,1,1
2,0.0101,3895546263509774583,3895546263509774583_1472865386,0.089512,0.000000,0.089512,Chrome,0,Madrid,Spain,...,0,0,0,0,0,0,0,1,0,1
3,0.0101,4763447161404445595,4763447161404445595_1472881213,0.090012,0.000000,0.090012,UC Browser,0,not available in demo dataset,Indonesia,...,0,0,0,0,0,0,1,0,0,1
4,0.0101,27294437909732085,27294437909732085_1472822600,0.088159,0.002538,0.088159,Chrome,0,not available in demo dataset,United Kingdom,...,0,0,0,0,0,0,0,1,0,1


In [32]:
test_df_true_false[cols_to_minmax_scale] = scaler.fit_transform(test_df_true_false[cols_to_minmax_scale])
test_df_true_false.head()

C:\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,device.browser,device.isMobile,geoNetwork.city,geoNetwork.country,...,device.operatingSystem Windows Phone,device.operatingSystem Xbox,device.operatingSystem iOS,geoNetwork.continent (not set),geoNetwork.continent Africa,geoNetwork.continent Americas,geoNetwork.continent Asia,geoNetwork.continent Europe,geoNetwork.continent Oceania,socialEngagementType Not Socially Engaged
0,0.022227,6167871330617112363,6167871330617112363_1508151024,0.276350,0.002193,0.276326,Chrome,0,(not set),Singapore,...,0,0,0,0,0,0,1,0,0,1
1,0.022227,0643697640977915618,0643697640977915618_1508175522,0.277393,0.000000,0.277368,Chrome,0,Zaragoza,Spain,...,0,0,0,0,0,0,0,1,0,1
2,0.022227,6059383810968229466,6059383810968229466_1508143220,0.276018,0.000000,0.275994,Chrome,0,not available in demo dataset,France,...,0,0,0,0,0,0,0,1,0,1
3,0.022227,2376720078563423631,2376720078563423631_1508193530,0.278159,0.000000,0.278134,Safari,0,Mountain View,United States,...,0,0,1,0,0,1,0,0,0,1
4,0.022227,2314544520795440038,2314544520795440038_1508217442,0.279176,0.000000,0.279152,Safari,0,San Jose,United States,...,0,0,0,0,0,1,0,0,0,1


In [54]:
def one_hot_encode_col_by_distribution(df, col):
    THRESHOLD = 0.01

    value_counts = df[col].value_counts()
    max_value_count = value_counts.max()
    percentages = (value_counts / value_counts.max())
    bigger_than_threshold = percentages > THRESHOLD
    values_to_keep = (bigger_than_threshold[bigger_than_threshold == True]).keys()
    
    df[col] = df[col].apply(lambda x: x if x in values_to_keep else 'Other')
    
    one_hot_encoded_cols = pd.get_dummies(df[col], prefix=col, prefix_sep=' ')
    df_without_original_col = df.drop(col, axis=1)
    return pd.concat([df_without_original_col, one_hot_encoded_cols], axis=1, sort=False)

def one_hot_encode_cols_by_distribution(df, cols):
    for col in cols:
        df = one_hot_encode_col_by_distribution(df, col)
    return df

cols_to_embed = ['device.browser', 'geoNetwork.city',
                'geoNetwork.country', 'geoNetwork.networkDomain',
                'trafficSource.medium', 'geoNetwork.subContinent', 'trafficSource.source']

train_df_wrangled = one_hot_encode_cols_by_distribution(train_df_true_false, cols_to_embed)
train_df_wrangled.head()

,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,device.isMobile,totals.bounces,totals.hits,totals.newVisits,...,geoNetwork.subContinent Western Europe,trafficSource.source (direct),trafficSource.source Other,trafficSource.source Partners,trafficSource.source analytics.google.com,trafficSource.source dfa,trafficSource.source google,trafficSource.source google.com,trafficSource.source mall.googleplex.com,trafficSource.source youtube.com
0,0.0101,1131660440785968503,1131660440785968503_1472830385,0.088405,0.000000,0.088405,0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
1,0.0101,377306020877927890,377306020877927890_1472880147,0.089979,0.000000,0.089979,0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
2,0.0101,3895546263509774583,3895546263509774583_1472865386,0.089512,0.000000,0.089512,0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
3,0.0101,4763447161404445595,4763447161404445595_1472881213,0.090012,0.000000,0.090012,0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
4,0.0101,27294437909732085,27294437909732085_1472822600,0.088159,0.002538,0.088159,0,0.0,0.0,NaN,...,0,0,0,0,0,0,1,0,0,0


In [55]:
test_df_wrangled = one_hot_encode_cols_by_distribution(test_df_true_false, cols_to_embed)
test_df_wrangled.head()

,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,device.isMobile,totals.bounces,totals.hits,totals.newVisits,...,geoNetwork.subContinent Western Europe,trafficSource.source (direct),trafficSource.source Other,trafficSource.source Partners,trafficSource.source analytics.google.com,trafficSource.source gdeals.googleplex.com,trafficSource.source google,trafficSource.source mall.googleplex.com,trafficSource.source sites.google.com,trafficSource.source youtube.com
0,0.022227,6167871330617112363,6167871330617112363_1508151024,0.276350,0.002193,0.276326,0,NaN,0.006012,NaN,...,0,0,0,0,0,0,1,0,0,0
1,0.022227,0643697640977915618,0643697640977915618_1508175522,0.277393,0.000000,0.277368,0,NaN,0.008016,0.0,...,0,0,0,0,0,0,1,0,0,0
2,0.022227,6059383810968229466,6059383810968229466_1508143220,0.276018,0.000000,0.275994,0,NaN,0.012024,0.0,...,1,0,0,0,0,0,1,0,0,0
3,0.022227,2376720078563423631,2376720078563423631_1508193530,0.278159,0.000000,0.278134,0,NaN,0.014028,0.0,...,0,0,0,0,0,0,1,0,0,0
4,0.022227,2314544520795440038,2314544520795440038_1508217442,0.279176,0.000000,0.279152,0,NaN,0.016032,0.0,...,0,0,0,0,0,0,1,0,0,0


In [67]:
train_columns_except_label = [col for col in train_df_wrangled.columns.values if col != LABEL]
test_columns = test_df_wrangled.columns.values

common_cols = list(set(train_columns_except_label) & set(test_columns))
cols_to_remove_train = [col for col in train_columns_except_label if col not in common_cols]
cols_to_remove_test = [col for col in test_columns if col not in common_cols]

train_df_uniform = train_df_wrangled.drop(cols_to_remove_train, axis=1)
train_df_uniform = train_df_uniform[[c for c in train_df_uniform if c not in [LABEL]] + [LABEL]]
train_df_uniform.head()

,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,device.isMobile,totals.bounces,totals.hits,totals.newVisits,...,geoNetwork.subContinent Western Asia,geoNetwork.subContinent Western Europe,trafficSource.source (direct),trafficSource.source Other,trafficSource.source Partners,trafficSource.source analytics.google.com,trafficSource.source google,trafficSource.source mall.googleplex.com,trafficSource.source youtube.com,totals.transactionRevenue
0,0.0101,1131660440785968503,1131660440785968503_1472830385,0.088405,0.000000,0.088405,0,0.0,0.0,0.0,...,1,0,0,0,0,0,1,0,0,NaN
1,0.0101,377306020877927890,377306020877927890_1472880147,0.089979,0.000000,0.089979,0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,NaN
2,0.0101,3895546263509774583,3895546263509774583_1472865386,0.089512,0.000000,0.089512,0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,NaN
3,0.0101,4763447161404445595,4763447161404445595_1472881213,0.090012,0.000000,0.090012,0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,NaN
4,0.0101,27294437909732085,27294437909732085_1472822600,0.088159,0.002538,0.088159,0,0.0,0.0,NaN,...,0,0,0,0,0,0,1,0,0,NaN


In [66]:
test_df_uniform = test_df_wrangled.drop(cols_to_remove_test, axis=1)
test_df_uniform.head()

,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,device.isMobile,totals.bounces,totals.hits,totals.newVisits,...,geoNetwork.subContinent Southern Europe,geoNetwork.subContinent Western Asia,geoNetwork.subContinent Western Europe,trafficSource.source (direct),trafficSource.source Other,trafficSource.source Partners,trafficSource.source analytics.google.com,trafficSource.source google,trafficSource.source mall.googleplex.com,trafficSource.source youtube.com
0,0.022227,6167871330617112363,6167871330617112363_1508151024,0.276350,0.002193,0.276326,0,NaN,0.006012,NaN,...,0,0,0,0,0,0,0,1,0,0
1,0.022227,0643697640977915618,0643697640977915618_1508175522,0.277393,0.000000,0.277368,0,NaN,0.008016,0.0,...,1,0,0,0,0,0,0,1,0,0
2,0.022227,6059383810968229466,6059383810968229466_1508143220,0.276018,0.000000,0.275994,0,NaN,0.012024,0.0,...,0,0,1,0,0,0,0,1,0,0
3,0.022227,2376720078563423631,2376720078563423631_1508193530,0.278159,0.000000,0.278134,0,NaN,0.014028,0.0,...,0,0,0,0,0,0,0,1,0,0
4,0.022227,2314544520795440038,2314544520795440038_1508217442,0.279176,0.000000,0.279152,0,NaN,0.016032,0.0,...,0,0,0,0,0,0,0,1,0,0


In [68]:
%%time
train_df_uniform.to_csv(OUT_TRAIN, index=False)
test_df_uniform.to_csv(OUT_TEST, index=False)

Wall time: 3min 2s
